# Evaluation

In this notebook we will demonstrate how to evaluate a set of generated mixes via objective metrics.

We will use the mixes generated from the [inference notebook](https://github.com/csteinmetz1/automix-toolkit/blob/main/notebooks/inference.ipynb), and we will objectively compare those mixes to the human-made groudn truth mixes.

The objective evaluation of mixes can be carried out through audio features that relate to the most common audio effects used during mixing. Since audio effects generally manipulate audio characteristics such as frequency content, dynamics, spatialization, timbre, or pitch, we can use audio features that are associated with these audio characteristics as a way to numerically evaluate mixes.

We can use the following audio features:
    
-**Spectral features** for EQ and reverberation: centroid, bandwidth, contrast, flatness, and roll-off

-**Spatialisation features** for panning: the Panning Root Mean Square (RMS)

-**Dynamic features** for dynamic range processors: RMS level, dynamic spread and crest factor

-**Loudness features**: the integrated loudness level (LUFS) and peak loudness

Note: This notebook assumes that you have already installed the `automix` package. 

In [ ]:
!pip install git+https://github.com/csteinmetz1/automix-toolkit

In [ ]:
import os
import glob
import torchaudio
import numpy as np

import IPython
import IPython.display as ipd
import matplotlib.pyplot as plt
import librosa.display

%matplotlib inline
%load_ext autoreload
%autoreload 2

plt.rcParams['figure.figsize'] = [10, 6]
plt.rcParams['figure.dpi'] = 100 

from automix.evaluation.utils_evaluation import get_features

# Drums mixing evaluation

We will evaluate two different trained models with a test sample from the ENST-drums subset.

Models: the Differentiable Mixing Console (DMC), and the MixWaveUNet.

In [ ]:
# then download and extract a drum multitrack from the test set
!wget https://huggingface.co/csteinmetz1/automix-toolkit/resolve/main/drums-test-rock.zip
!unzip -o drums-test-rock.zip

mix_target_path = "drums-test-rock/mix/dry_mix_066_phrase_rock_complex_fast_sticks.wav"
mix_auto_path_wun = "drums-test-rock/mix/dry_mix_066_phrase_rock_complex_fast_sticks_MixWaveUNet.wav"
mix_auto_path_dmc = "drums-test-rock/mix/dry_mix_066_phrase_rock_complex_fast_sticks_DMC.wav"

# Global Settings
SR = 44100
max_samples = 262144
start_sample = 0 * SR
end_sample = start_sample + max_samples

## Load the mixes

In [ ]:
fig, axs = plt.subplots(2, 1)

target_audio, sr = torchaudio.load(mix_target_path)
target_audio = target_audio[:, start_sample: end_sample]

librosa.display.waveshow(
    target_audio[0,:].numpy(), 
    x_axis='s', 
    sr=SR, 
    zorder=3, 
    label='human-made', 
    color='k', 
    ax=axs[0])

wun_audio, sr = torchaudio.load(mix_auto_path_wun)
wun_audio = wun_audio[:, start_sample: end_sample]
librosa.display.waveshow(
    wun_audio[0,:].view(-1).numpy(), 
    x_axis='s', 
    sr=SR, 
    zorder=3, 
    label='MixWaveUNet', 
    color='tab:blue', 
    ax=axs[0])
axs[0].grid(c="lightgray")
axs[0].legend()

librosa.display.waveshow(
    target_audio[0,:].numpy(), 
    x_axis='s', 
    sr=SR, 
    zorder=3, 
    label='human-made', 
    color='k', 
    ax=axs[1])

dmc_audio, sr = torchaudio.load(mix_auto_path_dmc)
dmc_audio = dmc_audio[:, start_sample: end_sample]
librosa.display.waveshow(dmc_audio[0,:].view(-1).numpy(), x_axis='s', sr=SR, zorder=3, label='DMC', color='tab:orange', ax=axs[1])
axs[1].grid(c="lightgray")
axs[1].legend()

## Compute the loudness, spectral, panning and dynamic features

In [ ]:
target_audio = target_audio.numpy()
wun_audio = wun_audio.numpy()
dmc_audio = dmc_audio.numpy()

print(target_audio.shape)
print(wun_audio.shape)


In [ ]:
wun_features = get_features(target_audio, wun_audio)
dmc_features = get_features(target_audio, dmc_audio)

In [ ]:
for k, i in wun_features.items():
    print(k, i)   

In [ ]:
for k, i in dmc_features.items():
    print(k, i)

In [ ]:
plt.bar(*zip(*wun_features.items()), color='c', label='MixWaveUNet')
plt.bar(*zip(*dmc_features.items()), color='m', label='DMC')
plt.axvline(2.5, 0, 1, linestyle='--', alpha=0.5, color='k', linewidth=0.75)
plt.axvline(10.5, 0, 1, linestyle='--', alpha=0.5, color='k', linewidth=0.75)
plt.axvline(15.5, 0, 1, linestyle='--', alpha=0.5, color='k', linewidth=0.75)
plt.xticks(rotation=-90)
plt.legend()
plt.show()